In [1]:
import os.path as osp
import sys
path = osp.dirname(osp.abspath(''))
sys.path.append(path)
sys.path.append(osp.join(path, "open_biomed"))
path

'/mnt/luoyz_dair/OpenBioMed'

BioMedGPT-10B is composed of a language model (BioMedGPT-LM-7B), a molecule encoder (GraphMVP), a protein encoder (ESM2-3B) and two modality adaptors. For inference, install `config.json`, `special_token_map.json`, `tokenizer_config.json`, `tokenizer.json`, `tokenizer.model` for BioMedGPT-LM-7B [here](https://huggingface.co/PharMolix/BioMedGPT-LM-7B) and put them under `OpenBioMed/ckpts/text_ckpts/biomedgpt-lm-7b`. Install `config.json`, `tokenizer_config.json`, `vocab.txt` for ESM2-3B [here](https://huggingface.co/facebook/esm2_t36_3B_UR50D) and put them under `OpenBioMed/ckpts/text_ckpts/biomedgpt-lm-7b`. Approximately 20GB GPU Memory is required to load BioMedGPT-10B with fp16.

In [2]:
# load biomedgpt model
import json
import torch
from open_biomed.utils import fix_path_in_config
from open_biomed.models.multimodal import BioMedGPTV

config = json.load(open("../configs/encoders/multimodal/biomedgptv.json", "r"))
fix_path_in_config(config, path)
print("Config: ", config)

device = torch.device("cuda:0")
config["network"]["device"] = device
model = BioMedGPTV(config["network"])
ckpt = torch.load("../ckpts/fusion_ckpts/biomedgpt_10b.pth")
model.load_state_dict(ckpt)
model = model.to(device)
model.eval()
print("Finish loading model")

/home/luoyz/.conda/envs/biomedgpt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Config:  {'name': 'biomedgptv', 'data': {'mol': {'modality': ['structure'], 'featurizer': {'structure': {'name': 'BaseGNN'}}}, 'protein': {'modality': ['structure'], 'featurizer': {'structure': {'name': 'transformertok', 'transformer_type': 'esm', 'model_name_or_path': '/mnt/luoyz_dair/OpenBioMed/ckpts/protein_ckpts/esm2-3b', 'max_length': 1024}}}}, 'network': {'mol': {'gin_hidden_dim': 300, 'gin_num_layers': 5, 'drop_ratio': 0.0, 'max_n_nodes': 256, 'freeze': True}, 'protein': {'model_name_or_path': '/mnt/luoyz_dair/OpenBioMed/ckpts/protein_ckpts/esm2-3b', 'lora': False, 'freeze': True, 'use_float16': True}, 'llm': {'model_name_or_path': '/mnt/luoyz_dair/OpenBioMed/ckpts/text_ckpts/biomedgpt-lm-7b', 'use_float16': True}}}


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/luoyz_dair/OpenBioMed/ckpts/protein_ckpts/esm2-3b/config.json'

In [5]:
# chat
from open_biomed.utils.chat_utils import Conversation

prompt_sys = "You are working as an excellent assistant in chemistry and molecule discovery. " + \
             "Below a human expert gives the representation of a molecule or a protein. Answer questions about it. "
chat = Conversation(
    model=model, 
    processor_config=config["data"], 
    device=device,
    system=prompt_sys,
    roles=("Human", "Assistant"),
    sep="###",
    max_length=2048
)
chat.append_molecule("CC(=CCC1=CC(=CC(=C1O)CC=C(C)C)/C=C/C(=O)C2=C(C=C(C=C2)O)O)C")
question = "Please describe this molecule."
print("Human: ", question)
chat.ask(question)
print("Assistant: ", chat.answer()[0])

Human:  Please describe this molecule.
Assistant:  The molecule is a member of the class of flavonoids that is 3-hydroxyflavone substituted by a 3-hydroxy-4-methoxyphenyl group at position 7. It has a role as a plant metabolite. It is a member of flavonoids, a member of 3-hydroxyflavones and a member of 3-hydroxyflavones. It derives from a 3-hydroxyflavone.


In [6]:
chat.reset()
chat.append_protein("MAKEDTLEFPGVVKELLPNATFRVELDNGHELIAVMAGKMRKNRIRVLAGDKVQVEMTPYDLSKGRINYRFK")
question = "What is the function of this protein?"
print("Human: ", question)
chat.ask(question)
print("Assistant: ", chat.answer()[0])

Human:  What is the function of this protein?
Assistant:  One of the essential components for the initiation of protein synthesis.  Stabilizes the binding of IF-2 and IF-3 on the 30S subunit to which N-formylmethionyl-tRNA(fMet) subsequently binds.  Helps modulate mRNA selection, yielding the 30S pre-initiation complex (PIC).  Upon addition of the 50S ribosomal subunit IF-1, IF-2 and IF-3 are released leaving the mature 70S translation initiation complex.
